In [ ]:
from IPython.display import HTML

import pysal as ps
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
%matplotlib inline

import folium
import random as RD


In [ ]:
def inline_map(map):
    """
    Embeds the HTML source of the map directly into the IPython notebook.
    
    This method will not work if the map depends on any files (json data). Also this uses
    the HTML5 srcdoc attribute, which may not be supported in all browsers.
    """
    map._build_map()
    return HTML('<iframe srcdoc="{srcdoc}" style="width: 100%; height: 510px; border: none"></iframe>'.format(srcdoc=map.HTML.replace('"', '&quot;')))

def embed_map(map, path="map.html"):
    """
    Embeds a linked iframe to the map into the IPython notebook.
    
    Note: this method will not capture the source of the map into the notebook.
    This method should work for all maps (as long as they use relative urls).
    """
    map.create_map(path=path)
    return HTML('<iframe src="files/{path}" style="width: 100%; height: 510px; border: none"></iframe>'.format(path=path))

## Binding Attribute Data to the Map

In [ ]:
import json

f = open(r'data/south.json')
q = json.load(f)
f.close()

In [ ]:
q.keys()

In [ ]:
features = q['features']
len(features)

In [ ]:
feature_0 = features[0]

In [ ]:
feature_0

In [ ]:
import pandas as pd
indices = []
values = []
for feature in features:
    indices.append(str(feature['properties']['FIPS']))
    values.append(feature['properties']['HR80'])

In [ ]:
df = pd.DataFrame({'HR80': values,
                   'FIPS': indices} )

In [ ]:

map_osm = folium.Map(location=[33.7550, -87.3900], zoom_start=5,)
map_osm.geo_json(geo_path='data/south.json',
                 key_on='feature.properties.FIPS',
                 data_out='data.json', data=df,
                 columns=['FIPS','HR80'],
                 fill_color='YlGnBu', fill_opacity=0.7,
                 line_opacity=0.2,
                 legend_name='Homicide Rate HR80')
embed_map(map_osm)


In [ ]:
y = np.array(df.HR80.tolist())

In [ ]:
bins= ps.Quantiles(y, 5).bins.tolist()
map_osm = folium.Map(location=[33.7550, -87.3900], zoom_start=5,)
map_osm.geo_json(geo_path='data/south.json',
                 key_on='feature.properties.FIPS',
                 data_out='data.json', data=df,
                 columns=['FIPS','HR80'],
                 fill_color='YlGnBu', fill_opacity=0.7,
                 line_opacity=0.2,
                 threshold_scale=bins[:-1],
                 legend_name='Homicide Rate HR80 (Quantiles k=5)')
embed_map(map_osm)

## Global Spatial Autocorrelation

In [ ]:
w = ps.queen_from_shapefile('data/south.shp')

In [ ]:
w.n

In [ ]:
w.transform = 'r'

In [ ]:
np.random.seed(12345)
mi_hr80 = ps.Moran(y, w)

In [ ]:
mi_hr80.p_sim

In [ ]:
mi_hr80.I

In [ ]:
mi_hr80.EI

In [ ]:
mi_hr80.p_norm

In [ ]:
yr = np.random.permutation(y)

In [ ]:
df['yrandom'] = yr

In [ ]:
type(y)

In [ ]:
bins= ps.Quantiles(y, 5).bins.tolist()
map_osm = folium.Map(location=[33.7550, -87.3900], zoom_start=5,)
map_osm.geo_json(geo_path='data/south.json',
                 key_on='feature.properties.FIPS',
                 data_out='data.json', data=df,
                 columns=['FIPS','HR80'],
                 fill_color='YlGnBu', fill_opacity=0.7,
                 line_opacity=0.2,
                 threshold_scale=bins[:-1],
                 legend_name='Homicide Rate HR80 (Quantiles k=5)')
embed_map(map_osm)

In [ ]:
bins= ps.Quantiles(y, 5).bins.tolist()
map_osm = folium.Map(location=[33.7550, -87.3900], zoom_start=5,)
map_osm.geo_json(geo_path='data/south.json',
                 key_on='feature.properties.FIPS',
                 data_out='data.json', data=df,
                 columns=['FIPS','yrandom'],
                 fill_color='YlGnBu', fill_opacity=0.7,
                 line_opacity=0.2,
                 threshold_scale=bins[:-1],
                 legend_name='Spatially Random Homicide Rate HR80 (Quantiles k=5)')
embed_map(map_osm)

In [ ]:
mi_hr80_random = ps.Moran(df['yrandom'],w)

In [ ]:
mi_hr80_random.I

In [ ]:
mi_hr80_random.p_sim

In [ ]:
ps.Moran??

In [ ]:
from scipy.stats import gaussian_kde

In [ ]:
density = gaussian_kde(mi_hr80.sim)
xs = np.linspace(mi_hr80.sim.min(),mi_hr80.sim.max(),200)
plt.plot(xs, density(xs))
plt.axvline(x=mi_hr80.I, color='r')
plt.xlabel('I')
plt.ylabel('f(I)')

In [ ]:
density = gaussian_kde(mi_hr80.sim)
xs = np.linspace(mi_hr80.sim.min(),mi_hr80.sim.max(),200)
plt.plot(xs, density(xs))
plt.axvline(x=mi_hr80.I, color='r')
plt.axvline(x=mi_hr80_random.I, color='g')
plt.xlabel('I')
plt.ylabel('f(I)')

## Getis Ord Global G

In [ ]:
wd = ps.G(y,w, permutations=999)

In [ ]:
wd.p_norm

In [ ]:
wd.p_sim